# Recommender System for a Medical Supplies Company

The project aims to drive incremental sales by implementing a recommender system. The process involves data preparation and modeling steps, beginning with handling missing data and identifying the most popular products by sales volume and revenue. The analysis will also include identifying the company with the highest purchase volume. Two types of recommendation systems will be developed: a popularity-based recommender and a matrix factorization-based recommender. Additionally, cosine similarity will be applied to product descriptions to analyze product relationships, enhancing personalization and boosting sales.

<img src='https://affordablemedmart.com/wp-content/uploads/2014/02/front-room-e1609957135987.jpg'>

### Import Dataset

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_column',500)
pd.set_option('display.max_row',500)

In [2]:
df = pd.read_csv('PBL 5 recommendation data.csv', encoding='iso-8859-1')

### EDA and Featue Engineering

In [3]:
df.head()

,Customers.id,Customers.fname,Customers.lname,Customers.company,Customers.create_date,Customers.status,Customers.mailing,Customers.reminders,Customers.tax_exempt,Customers.account_id,Customers.sales_rep,Customers.rewards,Customers.profile_id,Customers.last_modified,Customers.customer_type,Orders.id,Orders.customer_id,Orders.fname,Orders.lname,Orders.company,Orders.order_number,Orders.reorder_id,Orders.external_source,Orders.external_id,Orders.currency,Orders.sales_rep,Orders.subtotal,Orders.tax,Orders.shipping,Orders.coupon_id,Orders.coupon_amount,Orders.gift_id,Orders.gift_amount,Orders.fee_name,Orders.fee_amount,Orders.discount_name,Orders.discount_amount,Orders.total,Orders.balance_due,Orders.shipping_carrier,Orders.shipping_method,Orders.shipping_trans,Orders.shipping_flags,Orders.weight,Orders.tracking,Orders.payment_status,Orders.payment_date,Orders.payment_user,Orders.payment_type,Orders.payment_method,Orders.payment_amount,Orders.purchase_order,Orders.payment_id,Orders.payment_code,Orders.payment_ref,Orders.status,Orders.placed_date,Orders.updated_date,Orders.shipped_date,Orders.comments,Orders.notes,Orders.registry_id,Orders.gift_message,Orders.website,Orders.mailing,Orders.flags,Orders.partial_ship,Orders.customer_type,Order_Items.id,Order_Items.parent,Order_Items.product_id,Order_Items.product_name,Order_Items.attributes,Order_Items.attribute_names,Order_Items.attribute_prices,Order_Items.qty,Order_Items.price,Order_Items.cost,Order_Items.registry_item,Order_Items.related_id,Order_Items.reorder_frequency,Order_Items.account_id,Order_Items.flags,Products.id,Products.status,Products.product_type,Products.template,Products.vendor,Products.import_id,Products.name,Products.display_name,Products.menu_name,Products.list_price,Products.price,Products.sale_price,Products.cost,Products.flags,Products.left_flag,Products.right_flag,Products.last_modified,Products.taxable,Products.shopping_gtin,Products.shopping_brand,Products.shopping_mpn,Products.shopping_gender,Products.shopping_color,Products.shopping_age,Products.shopping_flags,Products.amazon_asin,Products.amazon_type,Products.amazon_item_type,Products.amazon_price,Products.google_shopping_id,Products.google_shopping_type,Products.google_shopping_cat,Products.google_adwords,Products.shopping_cat,Products.shopping_type,Products.pricegrabber_cat,Products.shopzilla_cat,Products.thefind_cat,Products.quickbooks_id,Products.qb_edit_sequence,Products.price_break_type,Products.price_breaks,Products.short_description,Products.long_description,Products.websites,Products.video,Products.audio,Products.seo_title,Products.seo_description,Products.seo_keywords,Products.seo_header,Products.seo_footer,Products.seo_url,Products.seo_category,Products.unit,Products.packaging,Products.display_packaging,Products.multiple,Products.length,Products.width,Products.height,Products.rx,Products.latex,Products.upc,Products.msds_link,Products.msds_label,Products.lit_link,Products.lit_label,Products.hcpcs,Products.case_qty,Products.markup,Products.override_markup,Products.notes,Products.import_flags,Products.map_price,Products.features_title,Products.warranty,Products.hygienic,Products.default_quantity,Products.user_size,Products.assembly,Products.installation,Products.shipping_length,Products.shipping_width,Products.shipping_height,Products.shipping_weight,Products.handling_time,Products.rotation_link,Products.google_shopping_label,Products.product_option,Products.size,Products.material,Products.arm_style,Products.leg_style,Products.seat_size,Products.family_id,Products.saved_status,Products.freight_cost
0,797,Christy,Dill,Company0,1426018724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1437764306,0.0,3758,797,Christy,Dill,Company0,3758,NaN,NaN,NaN,USD,NaN,57.20,0.0,9.95,13.0,2.86,NaN,NaN,NaN,NaN,NaN,NaN,64.29,NaN,fedex,11|Ground,NaN,NaN,NaN,5.7204E+14,3.0,1.426019e+09,NaN,authorize.net,NaN,64.29,NaN,6993607863,510142,NaN,1,1426019099,1.438868e+09,1.426101e+09,NaN,Insured By Eye4Fraud,NaN,NaN,NaN,NaN,NaN,NaN,0.0,528

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4194 entries, 0 to 4193
Columns: 181 entries, Customers.id to Products.freight_cost
dtypes: float64(98), int64(10), object(73)
memory usage: 5.8+ MB


In [5]:
df.columns

Index(['Customers.id', 'Customers.fname', 'Customers.lname',
       'Customers.company', 'Customers.create_date', 'Customers.status',
       'Customers.mailing', 'Customers.reminders', 'Customers.tax_exempt',
       'Customers.account_id',
       ...
       'Products.google_shopping_label', 'Products.product_option',
       'Products.size', 'Products.material', 'Products.arm_style',
       'Products.leg_style', 'Products.seat_size', 'Products.family_id',
       'Products.saved_status', 'Products.freight_cost'],
      dtype='object', length=181)

In [6]:
df.isnull().sum().sort_values(ascending=False)

Products.freight_cost             4194
Orders.mailing                    4194
Orders.gift_message               4194
Orders.registry_id                4194
Products.right_flag               4194
Orders.sales_rep                  4194
Products.markup                   4194
Orders.payment_ref                4194
Products.websites                 4194
Orders.purchase_order             4194
Orders.gift_id                    4194
Orders.gift_amount                4194
Products.price_breaks             4194
Products.seo_footer               4194
Products.default_quantity         4194
Orders.shipping_flags             4194
Products.price_break_type         4194
Orders.shipping_trans             4194
Orders.website                    4194
Products.amazon_type              4194
Order_Items.account_id            4194
Order_Items.attributes            4194
Order_Items.attribute_prices      4194
Products.menu_name                4194
Order_Items.registry_item         4194
Order_Items.related_id   

In [7]:
empty_data=df.isnull().sum()
sum_data=len(df)
pct=(empty_data/sum_data)*100
empty=pd.DataFrame({'empty_data':empty_data,'pct':pct})

In [8]:
empty.sort_values(by='pct',ascending=False)

,empty_data,pct
Products.freight_cost,4194,100.000000
Orders.mailing,4194,100.000000
Orders.gift_message,4194,100.000000
Orders.registry_id,4194,100.000000
Products.right_flag,4194,100.000000
Orders.sales_rep,4194,100.000000
Products.markup,4194,100.000000
Orders.payment_ref,4194,100.000000
Products.websites,4194,100.000000
Orders.purchase_order,4194,100.000000


There are so many null values in our dataframe. We will keep only columns have impact customer behavior.

In [9]:
df=df[['Customers.id', 'Customers.fname', 'Customers.lname', 'Customers.create_date', 'Customers.company', 'Orders.id', 'Orders.customer_id', 
       'Orders.total', 'Order_Items.product_id', 'Order_Items.qty', 'Products.id', 'Products.name', 'Products.shopping_brand', 'Products.price',
       'Products.cost']]                   

In [10]:
df.isnull().sum().sort_values(ascending=False)

Customers.company          3467
Products.shopping_brand     162
Products.price              158
Products.id                 157
Products.name               157
Products.cost               157
Order_Items.product_id       43
Customers.id                  0
Customers.fname               0
Customers.lname               0
Customers.create_date         0
Orders.id                     0
Orders.customer_id            0
Orders.total                  0
Order_Items.qty               0
dtype: int64

In [12]:
df['Customers.company']=df['Customers.company'].fillna(df['Customers.company'].mode()[0])
df['Products.shopping_brand']=df['Products.shopping_brand'].fillna(df['Products.shopping_brand'].mode()[0])
df['Products.price']=df['Products.price'].fillna(df['Products.price'].mean())
df['Products.id']=df['Products.id'].fillna(df['Products.id'].mode()[0])
df['Products.name']=df['Products.name'].fillna(df['Products.name'].mode()[0])
df['Products.cost']=df['Products.cost'].fillna(df['Products.cost'].mean())
df['Order_Items.product_id']=df['Order_Items.product_id'].fillna(df['Order_Items.product_id'].mode()[0])

In [13]:
df.isnull().sum().sort_values(ascending=False)

Customers.id               0
Customers.fname            0
Customers.lname            0
Customers.create_date      0
Customers.company          0
Orders.id                  0
Orders.customer_id         0
Orders.total               0
Order_Items.product_id     0
Order_Items.qty            0
Products.id                0
Products.name              0
Products.shopping_brand    0
Products.price             0
Products.cost              0
dtype: int64

### Popularity Based Recommendation

In [15]:
product_sales = df.groupby('Products.id').agg(max_volume=('Order_Items.qty', 'sum'),total_income=('Orders.total', 'sum')).reset_index()
product_sales = product_sales.merge(df[['Products.id', 'Products.name', 'Products.price']], on='Products.id', how='left')

In [17]:
most_popular_product = product_sales.loc[product_sales['total_income'].idxmax()]
most_popular_product

Products.id                           1842.0
max_volume                               910
total_income                        28252.48
Products.name     Emesis Bags,Blue,36.000 OZ
Products.price                     74.266043
Name: 1812, dtype: object

We found that the most popular product is Emesis Bags, Blue.

In [21]:
top_customers = df.groupby('Orders.customer_id').agg(total_purchasing=('Orders.total', 'sum')).reset_index()
top_customers = top_customers.merge(df[['Customers.id', 'Customers.fname', 'Customers.lname']], left_on='Orders.customer_id', 
                                    right_on='Customers.id', how='left')

In [24]:
max_purchasing_company = top_customers.sort_values(by='total_purchasing', ascending=False)
unique_max_purchasing_company = max_purchasing_company.drop_duplicates(subset='Customers.id')
unique_max_purchasing_company.head()

,Orders.customer_id,total_purchasing,Customers.id,Customers.fname,Customers.lname
1294,1043,15228.95,1043,Lee,Ceccotti
1597,1352,10007.48,1352,RoseAnn,Reyes
2404,2022,9502.36,2022,Att. Carmella,Monte
1946,1616,8530.28,1616,Jeffrey,Emerson
2068,1730,8452.78,1730,Melissa,Burton


The ranking of the top 5 companies with the highest purchases is as shown above.

In [25]:
popular_products = df.groupby('Products.id').agg(total_purhasing=('Order_Items.qty', 'sum'),total_income=('Orders.total', 'sum')).reset_index()
popular_products = popular_products.sort_values(by='total_purhasing', ascending=False)

In [26]:
customer_id = 1
customer_orders = df[df['Orders.customer_id'] == customer_id]['Order_Items.product_id'].unique()
recommended_products = popular_products[~popular_products['Products.id'].isin(customer_orders)]
print(recommended_products.head(5))

     Products.id  total_purhasing  total_income
599       1842.0              910      28252.48
600       1846.0              396      13442.16
662       2107.0              228      13606.61
566       1672.0              220       5093.30
557       1648.0              133       4987.50


We provided recommendations from popular products using this customer's previous purchases based on their customer ID.

### Matrix Factorization Recommendation

In [27]:
from sklearn.decomposition import TruncatedSVD

interaction_matrix = df.pivot_table(index='Orders.customer_id', columns='Order_Items.product_id', values='Order_Items.qty', fill_value=0)

In [28]:
interaction_matrix

Order_Items.product_id,1.0,11.0,14.0,15.0,19.0,20.0,22.0,26.0,28.0,29.0,30.0,35.0,62.0,64.0,65.0,74.0,76.0,77.0,79.0,80.0,83.0,86.0,87.0,89.0,96.0,100.0,105.0,111.0,123.0,132.0,137.0,139.0,146.0,147.0,148.0,153.0,155.0,163.0,166.0,167.0,168.0,169.0,174.0,176.0,177.0,183.0,210.0,223.0,226.0,227.0,234.0,236.0,243.0,245.0,248.0,250.0,251.0,253.0,254.0,255.0,256.0,259.0,260.0,263.0,264.0,265.0,266.0,267.0,269.0,270.0,271.0,273.0,274.0,276.0,278.0,279.0,285.0,297.0,302.0,303.0,305.0,306.0,310.0,312.0,318.0,320.0,321.0,322.0,326.0,327.0,328.0,329.0,331.0,332.0,333.0,338.0,342.0,344.0,345.0,346.0,349.0,350.0,352.0,355.0,359.0,361.0,365.0,367.0,369.0,370.0,373.0,374.0,375.0,376.0,380.0,381.0,383.0,384.0,385.0,387.0,388.0,389.0,390.0,391.0,392.0,396.0,397.0,400.0,404.0,405.0,406.0,408.0,409.0,412.0,414.0,415.0,416.0,417.0,421.0,425.0,426.0,428.0,430.0,432.0,433.0,434.0,439.0,468.0,487.0,493.0,509.0,531.0,533.0,535.0,536.0,539.0,551.0,561.0,562.0,563.0,564.0,565.0,566.0,570.0,574.0,575.0,577.0,578.0,580.0,583.0,584.0,587.0,595.0,597.0,599.0,600.0,602.0,604.0,605.0,606.0,612.0,615.0,639.0,649.0,716.0,724.0,732.0,734.0,745.0,747.0,748.0,749.0,750.0,751.0,758.0,769.0,773.0,774.0,776.0,782.0,784.0,785.0,788.0,789.0,794.0,795.0,799.0,802.0,811.0,812.0,814.0,815.0,816.0,817.0,821.0,822.0,823.0,826.0,827.0,828.0,829.0,830.0,831.0,835.0,836.0,837.0,839.0,840.0,841.0,842.0,844.0,845.0,846.0,848.0,850.0,851.0,852.0,853.0,857.0,858.0,859.0,860.0,861.0,862.0,863.0,865.0,870.0,871.0,872.0,874.0,...,15646.0,15649.0,15650.0,15653.0,15656.0,15705.0,15714.0,15746.0,15753.0,15757.0,15772.0,15774.0,15775.0,15776.0,15803.0,15809.0,15846.0,15849.0,15855.0,15870.0,15882.0,15903.0,15933.0,15947.0,15980.0,16031.0,16034.0,16045.0,16075.0,16099.0,16168.0,16187.0,16201.0,16220.0,16224.0,16232.0,16237.0,16240.0,16252.0,16256.0,16263.0,16286.0,16287.0,16345.0,16430.0,16474.0,16476.0,16479.0,16494.0,16500.0,16512.0,16522.0,16525.0,16531.0,16547.0,16558.0,16561.0,16566.0,16573.0,16575.0,16593.0,16599.0,16607.0,16632.0,16639.0,16642.0,16643.0,16645.0,16647.0,16656.0,16658.0,16660.0,16661.0,16662.0,16673.0,16683.0,16687.0,16689.0,16690.0,16693.0,16699.0,16717.0,16729.0,16743.0,16751.0,16776.0,16788.0,16831.0,16842.0,16843.0,16856.0,16858.0,16859.0,16880.0,16892.0,16893.0,16894.0,16904.0,16905.0,16909.0,16910.0,16913.0,16936.0,16956.0,16986.0,17012.0,17014.0,17021.0,17026.0,17033.0,17038.0,17051.0,17054.0,17061.0,17063.0,17066.0,17070.0,17072.0,17075.0,17076.0,17107.0,17126.0,17130.0,17134.0,17136.0,17142.0,17146.0,17147.0,17148.0,17149.0,17175.0,17179.0,17215.0,17230.0,17321.0,17335.0,17373.0,17465.0,17595.0,17651.0,17705.0,17766.0,17767.0,17812.0,17813.0,17815.0,17817.0,17818.0,17839.0,17841.0,17844.0,17848.0,17849.0,17850.0,18018.0,18024.0,18028.0,18031.0,18109.0,18122.0,18124.0,18126.0,18127.0,18140.0,18141.0,18186.0,18223.0,18230.0,18231.0,18232.0,18233.0,18235.0,18248.0,18252.0,18253.0,18277.0,18286.0,18318.0,18327.0,18338.0,18342.0,18348.0,18354.0,18363.0,18368.0,18371.0,18376.0,18401.0,18409.0,18410.0,18411.0,18414.0,18435.0,18484.0,18496.0,18509.0,18511.0,18513.0,18514.0,18517.0,18518.0,18522.0,18524.0,18534.0,18535.0,18548.0,18553.0,18554.0,18582.0,18604.0,18634.0,19048.0,19217.0,19447.0,20214.0,20216.0,20350.0,20954.0,21063.0,21370.0,21419.0,21639.0,21919.0,21920.0,21921.0,21950.0,24400.0,24416.0,24418.0,24422.0,24644.0,24699.0,24824.0,24845.0,24873.0,24887.0,24950.0,25001.0,25003.0,25107.0,25170.0,25269.0,25356.0,25527.0,25558.0,25612.0,25694.0,25908.0,25920.0,26175.0
Orders.customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

We printed the interaction matrix using matrix factorization above.

### KNN Recommendation

By using cosine similarity, we can extract each product's vector and determine the distance between these vectors in space. If the distance between vectors is small, we can say that the products are similar to each other.

In [29]:
# Example
products = [
    ['product_id_1', [0.2, 0.4, 0.5], 10],  
    ['product_id_2', [0.1, 0.3, 0.6], 20],
    ['product_id_3', [0.5, 0.7, 0.2], 5],
]

In [30]:
from scipy import spatial

def compute_distance(a, b):
    genre_distance = spatial.distance.cosine(a[1], b[1])
    popularity_distance = abs(a[2] - b[2])
    return genre_distance + popularity_distance


In [32]:
similar_products = sorted(
    [(p[0], compute_distance(products[0], p)) for p in products if p[0] != 'product_id_1'],
    key=lambda x: x[1]
)[:5]

print(similar_products)

[('product_id_3', 5.189808506333067), ('product_id_2', 10.032908639019512)]


Let's do it for our dataframe.

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
sparse_matrix=vect.fit_transform(df)

In [36]:
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, columns=vect.get_feature_names_out())

In [37]:
df.head()

,company,cost,create_date,customer_id,customers,fname,id,lname,name,order_items,orders,price,product_id,products,qty,shopping_brand,total
0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
from sklearn.metrics.pairwise import cosine_similarity
pd.DataFrame(cosine_similarity(df,df))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
1,0.5,1.0,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.5,0.5,1.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.5,0.5,0.5,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.5,0.5,0.5,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.5,0.0,0.0,0.0,0.0,1.0,0.5,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.5,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.5,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,0.0,0.0,0.0,0.0,0.0


### Conclusion

This project developed three recommendation systems for a medical supplies company: a popularity-based recommender, a matrix factorization-based recommender, and a K-Nearest Neighbors (KNN)-based recommender. Using Pandas and NumPy for data manipulation and preprocessing, the system identifies the most popular products and top-purchasing customers. The popularity-based and matrix factorization models, implemented with Scikit-Learn, provide suggestions based on sales volume and customer history. Meanwhile, the KNN recommender uses SciPy to calculate cosine similarity between product vectors, identifying similar items by vector distances for more accurate recommendations. Additionally, CountVectorizer from Scikit-Learn was used for text vectorization to enhance product relationship analysis. These systems ultimately aim to improve customer satisfaction and drive incremental sales through tailored recommendations.